## Repeated measures ANOVA with control group

### 1. Introduction
So far we have counted on having the ideal conditions for performing ANOVA: our groups were equal in number and, more importantly, our dataset was complete. We didn't have any missing values. As you may imagine, such perfect conditions are rather rare in evaluation research, particularly in longitudinal designs because several participants drop out. 

This scenario brings up challenges that ANOVA-related analysis approaches can't deal with in a straightforward way. ANOVA would solve the missing values problem by removing of cases with missing values. Remember, however, that measurements re costly and a small sample reduces statistical power. You or the company/hospital/practice spent so much time testing participants and you do not want to dispose of valuable data that has been acquired. Instead, you can use an alternative to ANOVA that is able to reliably model the information that is available. 

MLMs are not only great because they can deal with missing values, but also because:

* MLMs can estimate individual differences in change via the variance of random slopes. Remember: Earlier, researchers calculated the difference score to investigate individual differences in change, but the approach is biased by **variance restriction of the difference score**.

* MLMs also estimate individual differences in starting values (pre-intervention scores).

* The fixed effects will provide an estimate of the average starting value of the outcome and its change. 

* Interaction effects between levels (i.e., cross-level interactions) will inform us whether the change and individual differences in change depend on the group (treatment or control) and on further person specific variables like emotional intelligence in our example (variable "EM" in data frame). 

In the following exercise, we apply the `lmer()` function that you learned last semester. We fit a model by regressing depression scores onto *time*. As the *random* part of the function, we include "id" to denote the observational units within which the measurements are clustered (contextual variable). 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

### 2. Computation
**Summary:**

`import statsmodels.formula.api as smf`

`smf.mixedlm(formula="Dependent variable ~ fixed-effects terms", data=data.frame, groups=data.frame["Level-2 indicator variable"])`

A. **Fixed-Effects Terms**: All overall effects and interactions, i.e.,

    `predictor 1 + predictor 2 + predictor 1 * predictor 2...`

    Note that these predictors can be **Level-1** or **Level-2** variables.

B. **Random-Effects Terms**:

    In `statsmodels.formula.api.mixedlm()`, random effects are primarily handled through the `groups` argument, which specifies the Level-2 indicator variable. Random slopes and intercepts are not specified within the formula itself as they are in `lmer`.

    i. Only intercepts but not the slopes are random:

        This is achieved by using the `groups` argument. For example, if your Level-2 indicator variable is named "id", you would use `groups=data.frame["id"]`. This tells `mixedlm` to estimate a random intercept for each unique value in the "id" column.

    ii. Only slopes but not intercepts are random:

        `statsmodels.formula.api.mixedlm` does not directly support specifying only random slopes without random intercepts in the same way that `lmer` does with `(0 + Level-1 variable | Level-2 indicator variable)`. If you need to fit a model with only random slopes and no random intercepts, you would need to implement this by manually constructing the random effects design matrix and using the `random_effects` argument of `mixedlm`, which is an advanced use case.

    iii. Both slopes and intercepts are random:

        `statsmodels.formula.api.mixedlm` does not directly support the `(1 + Level-1 variable | Level-2 indicator variable)` syntax from `lmer`. Similiar to the 'only slopes' case, achieving random slopes and intercepts requires manual construction of the random effects design matrix and using the `random_effects` argument of `mixedlm`.

In [28]:
# Read the CSV file using a relative path
dep_long = pd.read_csv("../MLM_and_CSM/Datasets/dep_long.csv")

# Convert specific columns to categorical data types
dep_long['id'] = dep_long['id'].astype('category')


# Reorder the levels of the 'time' and 'group' factor
dep_long['time'] = pd.Categorical(dep_long['time'], categories=['pre', 'post'], ordered=True)
dep_long['group'] = pd.Categorical(dep_long['group'], categories=[0, 1], ordered=True)

# Fit the linear fixed effects model
formula = "score ~ time"
mu = smf.mixedlm(formula, dep_long, groups=dep_long["id"]).fit()
print(mu.summary())

          Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: score    
No. Observations:   200     Method:             REML     
No. Groups:         100     Scale:              321.1341 
Min. group size:    2       Log-Likelihood:     -861.2596
Max. group size:    2       Converged:          Yes      
Mean group size:    2.0                                  
---------------------------------------------------------
              Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
---------------------------------------------------------
Intercept     60.830    1.832 33.202 0.000  57.239 64.421
time[T.post] -14.190    2.534 -5.599 0.000 -19.157 -9.223
Group Var     14.539    1.964                            



This model estimates the **fixed intercept and slope**, (average depression score at the pre-therapy time point and average change in depression from pre- to post-therapy time point). The estimates are 60.830 for the intercept and  -14.190 for the slope, respectively.

We can add a second, level two (between-person) predictor to the model, indicating whether or not an individual belonged to the intervention group. We Extend the model "mu" by adding the "group" variable to the fixed part of the formula function. Please include **only** main effects. Name the model "mu2".

In [29]:
# Fit the linear fixed effects model
formula = "score ~ time + group"
mu = smf.mixedlm(formula, dep_long, groups=dep_long["id"]).fit()
print(mu.summary())

          Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: score    
No. Observations:   200     Method:             REML     
No. Groups:         100     Scale:              289.8089 
Min. group size:    2       Log-Likelihood:     -844.5104
Max. group size:    2       Converged:          Yes      
Mean group size:    2.0                                  
---------------------------------------------------------
              Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
---------------------------------------------------------
Intercept     67.675    2.085 32.458 0.000  63.589 71.761
time[T.post] -14.190    2.408 -5.894 0.000 -18.909 -9.471
group[T.1]   -13.690    2.408 -5.686 0.000 -18.409 -8.971
Group Var      0.000    1.955                            



c:\Users\danie\Documents\envPython\psy112\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


This model adds the effect of the variable `group` as a fixed effect (-13.690). It can be interpreted as the estimated group difference on the **overall** depression. This is the difference in depression between groups, as demonstrated here:

In [30]:
# Group by 'group' and calculate the mean of 'score' for each group
means_gr = dep_long.groupby('group')['score'].mean().reset_index()
means_gr.columns = ['group', 'mean_group']

# Print the means for each group
print(means_gr)

# Calculate the difference between the means of the groups
group_diff = means_gr['mean_group'].iloc[1] - means_gr['mean_group'].iloc[0]

# Print the difference
print(group_diff)

  group  mean_group
0     0       60.58
1     1       46.89
-13.689999999999998


Thus far, we  modeled the main effects (i.e., the effect of time and group). However, because we are interested in knowing how the change in depression from pre- to post-therapy differs between the groups (interaction effect), the next step is to include the **cross-level interaction** term between these two variables (time and group). Remember, the cross-level interaction means that a between-person variable (group) is interacting with a within-person variable (time). More generally, the cross-level interaction refers to any scenario where variables of different levels interact. 

In [35]:
# Fit the linear fixed effects model
formula = "score ~ time + group + time*group"
mu = smf.mixedlm(formula, dep_long, groups=dep_long["id"]).fit()
print(mu.summary())

                Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     score    
No. Observations:       200         Method:                 REML     
No. Groups:             100         Scale:                  246.0902 
Min. group size:        2           Log-Likelihood:         -827.8411
Max. group size:        2           Converged:              Yes      
Mean group size:        2.0                                          
---------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------
Intercept                61.480    2.246 27.379 0.000  57.079  65.881
time[T.post]             -1.800    3.137 -0.574 0.566  -7.949   4.349
group[T.1]               -1.300    3.176 -0.409 0.682  -7.524   4.924
time[T.post]:group[T.1] -24.780    4.437 -5.585 0.000 -33.476 -16.084
Group Var                 6.036    1

Note that the output now provides an effect named "group1:timepost", which equals -6.195 in our example. This effect indicates the difference in change over time across groups. Since the  `group` variable is dummy coded with 0 and 1, the negative coefficient means that as the group value increases from control (coded 0) to treatment (coded 1), the change over time in depression decreases by about 6.2 scale points. The "timepost" estimate is now -1.800. Since the group effect (change from 0 to 1) on change was already accounted for by the interaction term, -7.095 indicates the difference between post- and pre-measurements when ignoring the group effect. That is, for the control group.

This cross-level interaction term is essential to answer the question about the magnitude and statistical significance of the group effect on the depression change from the pre- to the post-measurements.

We can even proceed to add EM as predictor:

In [36]:
# Fit the linear fixed effects model
formula = "score ~ time + group + time*group + EM"
mu = smf.mixedlm(formula, dep_long, groups=dep_long["id"]).fit()
print(mu.summary())

                Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     score    
No. Observations:       200         Method:                 REML     
No. Groups:             100         Scale:                  246.0841 
Min. group size:        2           Log-Likelihood:         -828.0735
Max. group size:        2           Converged:              Yes      
Mean group size:        2.0                                          
---------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------
Intercept                57.190    3.774 15.155 0.000  49.794  64.586
time[T.post]             -1.800    3.137 -0.574 0.566  -7.949   4.349
group[T.1]               -1.227    3.168 -0.387 0.698  -7.436   4.982
time[T.post]:group[T.1] -24.780    4.437 -5.585 0.000 -33.476 -16.084
EM                        0.165    0

As in model "mu2", the **fixed** estimate for EM indicates its effect on the **overall** depression score. Because "EM" is a continuous variable, its estimate (0.16) is interpreted as a regression weight - the expected change in depression given a one-unit increase in EM.

Note: If you wanted to test whether the change in depression differs depending on EM, a further **cross-level interaction** between time and EM needs to be added to the model. We don't go over such a model, but the syntax to run it would be: 

In [37]:
# Fit the linear fixed effects model
formula = "score ~ time + group + time*group + EM + time*EM"
mu = smf.mixedlm(formula, dep_long, groups=dep_long["id"]).fit()
print(mu.summary())

                Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     score    
No. Observations:       200         Method:                 REML     
No. Groups:             100         Scale:                  247.3711 
Min. group size:        2           Log-Likelihood:         -828.3814
Max. group size:        2           Converged:              Yes      
Mean group size:        2.0                                          
---------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------
Intercept                59.275    4.815 12.312 0.000  49.839  68.712
time[T.post]             -5.970    6.753 -0.884 0.377 -19.205   7.266
group[T.1]               -1.263    3.172 -0.398 0.691  -7.480   4.955
time[T.post]:group[T.1] -24.709    4.450 -5.553 0.000 -33.431 -15.988
EM                        0.085    0

We hope that by now you are aware of the several advantages of MLM compared to more traditional approaches. Keep in mind that this is only a brief introduction to MLM in the context of longitudinal data analysis.